In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nfl-big-data-bowl-2026-prediction/test_input.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/test.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/nfl_inference_server.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/nfl_gateway.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/__init__.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/templates.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/base_gateway.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/relay.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/__init__.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/generated/kaggle_evaluation_pb2.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/generated/kaggle_evaluati

# NFL Big Data Bowl 2026 – Player Movement Prediction Project

**1. IMPORTS**

*Purpose:*
All required Python libraries are imported here, including data manipulation, numerical computation, machine learning frameworks, and deep learning libraries.

*Libraries used and reasoning:*

- pandas & numpy: Efficient data manipulation and numerical computation.

- torch & nn: For building and training the LSTM sequence model.

- xgboost, lightgbm, catboost: For tree-based residual models that refine LSTM outputs.

- os, glob, pickle: For file handling and model loading.

*Why this approach:*
Using separate imports and modularizing them makes the code easier to maintain, debug, and ensures all dependencies are listed upfront.

**2. FEATURE ENGINEERING FUNCTIONS**

*Purpose:*
Transform raw player tracking data into meaningful features for model input.

*Key logic:*

- Input data contains positions, velocities, accelerations, and other metrics per player per frame.

- Additional features like relative distance to the line of scrimmage, players’ orientation, and team identifiers were computed.

- Feature selection focused on columns used consistently across tree and LSTM models (TREE_FEATURE_COLS).

*Why this method:*
Proper feature engineering captures the spatial and temporal context for player movement, enabling both LSTM and tree models to make better predictions.

**3. MODEL ARCHITECTURE (LSTM)**

*Purpose:*
Predict the future positions of players over a sequence of frames.

*Model details:*

Encoder-Decoder LSTM with:

- Encoder: processes historical player trajectories.

- Decoder: predicts next positions frame by frame.

- Linear layer: maps hidden states to (x, y) coordinates.

- Handles variable sequence lengths using pack_padded_sequence.

- Uses last observed (x, y) as input for next frame prediction.

*Why LSTM:*
Player trajectories are sequential and time-dependent. LSTMs can capture temporal dependencies better than tree models alone.

**4. UTILITY FUNCTIONS**

*Purpose:*
Handle data preprocessing and batching for models.

*Functions include:*

- pad_groups(): Pads sequences to maximum length in batch for LSTM.

- make_groups_meta(): Organizes player trajectories by nfl_id and generates metadata for reconstructing predictions.

- Model loading utilities: load_lstm_model(), load_xgb_model(), load_lgb_model(), load_cat_model().

*Why this approach:*

Ensures compatibility between variable-length input sequences and fixed-size batch processing in PyTorch.

Simplifies model deployment by providing functions to load trained models safely.

**5. predict_play() — Multi-frame inference (LSTM + Residual Tree Models)**

*Purpose:*
Predict player positions over multiple frames using LSTM and optionally refine with tree-based residuals.

*Logic:*

- Generate features per player and batch into sequences.

- Run LSTM to predict (x, y) for T_out frames.

*If tree models exist:*

- Predict residuals for (x, y) using last input frame features.

- Ensemble residuals from XGBoost, LightGBM, and CatBoost using predefined weights.

- Apply residuals to LSTM output.

- Clip predicted coordinates to field boundaries.

*Why this method:*

- LSTM captures the sequential patterns in player movement.

- Tree models correct small systematic errors (residuals) using learned relationships from historical data.

- Ensures physically valid predictions.

**6. predict_one_play()**

*Purpose:*
Simplifies single-play prediction by integrating LSTM and tree model predictions.

*Logic:*

- Accepts a single play DataFrame.

- Calls predict_play() with preloaded models.

- Returns the final output in required schema: (game_id, play_id, nfl_id, frame_id, x, y).

*Why this method:*
Modularizes prediction for single-play use, which is necessary for the Kaggle evaluation loop or local testing.

**7. kaggle_main_offline() — Offline Prediction Loop**

*Purpose:*
Run predictions for the Kaggle Big Data Bowl using local CSV inputs, without requiring the Kaggle-specific nflrush environment.

*Logic:*

- Loads LSTM and tree-based residual models (XGBoost, LightGBM, CatBoost if available).

- Reads the test dataset directly from test_input.csv.

- Applies predict_one_play() to generate predicted player positions for all plays.

- Fills missing predictions with default values (0.0) to ensure no NaNs in the submission.

- Merges predictions with required Kaggle columns (game_id, play_id, nfl_id, frame_id).

- Saves the final predictions to a CSV file (submission.csv) for direct Kaggle submission.

*Why this method:*

- Fully offline and internet-free, compliant with Kaggle submission rules.

- Modular, easy to maintain, and flexible for testing with different model combinations.

- Avoids dependency on environment-specific packages while ensuring correct submission format.

**8. main / Local Testing**

*Purpose:*

Enable local execution and validation of the full prediction pipeline before submission.

*Logic:*

- Loads the full test dataset and model artifacts.

- Runs predict_one_play() on all test plays.

- Generates predictions in the Kaggle-required format.

- Saves or previews the output locally for debugging and performance checks.

*Why this approach:*

- Allows full validation of sequence and residual model predictions offline.

- Eliminates the risk of runtime errors due to missing environment-specific packages.

- Enables iterative performance tuning and debugging before submission.

**Summary**

- The project combines an LSTM model for sequential player movement prediction with tree-based residual models for accuracy enhancement.

- Modular architecture separates data preprocessing, model inference, and submission logic.

- Residual models correct errors from the LSTM predictions, improving overall performance.

- Local CSV-based workflow ensures safe, flexible, and reproducible testing and submission.

This structure is fully compatible with Kaggle Big Data Bowl rules and does not require internet access or nflrush.

# 1. IMPORTS

In [2]:
import os
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

# Optional tree libraries
try:
    import xgboost as xgb
except Exception:
    xgb = None

try:
    import lightgbm as lgb
except Exception:
    lgb = None

try:
    import catboost as cb
except Exception:
    cb = None

In [3]:
DEVICE = torch.device("cpu")

# If you have model artifacts, put them under /kaggle/input/... or change these paths.
LSTM_MODEL_PATH = "/kaggle/input/nfl-big-data-bowl-2026-prediction/best_lstm.pth"
XGB_MODEL_PATH = "/kaggle/input/nfl-big-data-bowl-2026-prediction/best_xgb.json"
LGB_MODEL_PATH = "/kaggle/input/nfl-big-data-bowl-2026-prediction/best_lgb.txt"
CAT_MODEL_PATH = "/kaggle/input/nfl-big-data-bowl-2026-prediction/best_cat.cbm"

ENSEMBLE_WEIGHTS = {"xgb": 0.3, "lgb": 0.3, "cat": 0.4}

# -------------------------
# Utility / FE functions
# -------------------------
def to_inches(h):
    try:
        ft, inch = str(h).split("-")
        return int(ft) * 12 + int(inch)
    except Exception:
        return 72


# 2. FEATURE ENGINEERING FUNCTIONS

In [4]:
def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    """Create physics-based features used by LSTM / tree models."""
    df = df.copy()

    # ensure required columns
    for col in ['x', 'y', 's', 'a', 'o', 'dir', 'player_height', 'player_weight', 'frame_id', 'nfl_id', 'game_id', 'play_id']:
        if col not in df.columns:
            # don't override required id columns if not present; but set defaults for FE
            if col in ['x', 'y', 's', 'a', 'o', 'dir']:
                df[col] = 0.0

    # heights & weight
    df['height_inches'] = df.get('player_height', '0').apply(to_inches).fillna(72)
    df['weight_lbs'] = pd.to_numeric(df.get('player_weight', 200), errors='coerce').fillna(200)

    # BMI
    df['bmi'] = (df['weight_lbs'] / (df['height_inches']**2 + 1e-6)) * 703.0

    # directions -> vectors
    dir_rad = np.radians(pd.to_numeric(df['dir'], errors='coerce').fillna(0.0))
    df['heading_x'] = np.sin(dir_rad)
    df['heading_y'] = np.cos(dir_rad)

    orient_rad = np.radians(pd.to_numeric(df['o'], errors='coerce').fillna(0.0))
    df['orient_x'] = np.sin(orient_rad)
    df['orient_y'] = np.cos(orient_rad)

    dcol = pd.to_numeric(df['dir'], errors='coerce').fillna(0.0)
    ocol = pd.to_numeric(df['o'], errors='coerce').fillna(0.0)
    diff = np.abs(dcol - ocol)
    df['dir_orient_diff'] = np.minimum(diff, 360 - diff)

    s = pd.to_numeric(df['s'], errors='coerce').fillna(0.0)
    a = pd.to_numeric(df['a'], errors='coerce').fillna(0.0)

    df['velocity_x'] = s * df['heading_x']
    df['velocity_y'] = s * df['heading_y']
    df['acceleration_x'] = a * df['heading_x']
    df['acceleration_y'] = a * df['heading_y']

    df['speed_squared'] = s**2
    df['accel_magnitude'] = np.sqrt(df['acceleration_x']**2 + df['acceleration_y']**2)

    df['momentum_x'] = df['weight_lbs'] * df['velocity_x']
    df['momentum_y'] = df['weight_lbs'] * df['velocity_y']
    df['momentum_magnitude'] = np.sqrt(df['momentum_x']**2 + df['momentum_y']**2)

    df['kinetic_energy'] = 0.5 * df['weight_lbs'] * df['speed_squared']

    df = df.replace([np.inf, -np.inf], 0.0).fillna(0.0)
    return df


TREE_FEATURE_COLS = [
    "x","y","s","a","o","dir",
    "heading_x","heading_y",
    "velocity_x","velocity_y",
    "acceleration_x","acceleration_y",
    "dir_orient_diff",
    "height_inches","weight_lbs","bmi",
    "speed_squared","accel_magnitude",
    "momentum_x","momentum_y","momentum_magnitude","kinetic_energy"
]


# 3. MODEL ARCHITECTURE (LSTM)

In [5]:
class EncoderDecoderLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, dropout=0.1):
        super().__init__()
        self.encoder = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers,
                               batch_first=True, dropout=dropout)
        self.decoder = nn.LSTM(2, hidden_dim, num_layers=num_layers,
                               batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, 2)

    def forward(self, enc_X, enc_lens, T_out=10):
        # enc_X shape: (B, T, F)
        packed = nn.utils.rnn.pack_padded_sequence(
            enc_X, enc_lens.cpu().numpy(), batch_first=True, enforce_sorted=False
        )
        _, (h_n, c_n) = self.encoder(packed)
        h, c = h_n, c_n

        # build last observed x,y for each sequence
        B = enc_X.size(0)
        last_xy = []
        for i, L in enumerate(enc_lens):
            Li = int(L.item())
            last_xy.append(enc_X[i, max(0, Li-1), :2])
        dec_in = torch.stack(last_xy, dim=0).unsqueeze(1)  # (B,1,2)

        preds = []
        for t in range(T_out):
            out, (h, c) = self.decoder(dec_in, (h, c))
            xy = self.fc(out.squeeze(1))
            preds.append(xy.unsqueeze(1))
            dec_in = xy.unsqueeze(1).detach()
        preds = torch.cat(preds, dim=1)  # (B, T_out, 2)
        return preds



# 4. UTILITY FUNCTIONS (padding, grouping, loading models)

In [6]:
# -------------------------
# Padding & grouping utilities
# -------------------------
def pad_groups(groups):
    """Pad list of (T_i, F) arrays to (B, T_max, F) and return torch tensors + lens"""
    B = len(groups)
    F = groups[0].shape[1]
    T_max = max(g.shape[0] for g in groups)

    Xp = np.zeros((B, T_max, F), dtype=np.float32)
    lens = np.zeros((B,), dtype=np.int64)
    for i, g in enumerate(groups):
        T = g.shape[0]
        Xp[i, :T, :] = g
        lens[i] = T
    Xt = torch.tensor(Xp, dtype=torch.float32, device=DEVICE)
    lens_t = torch.tensor(lens, dtype=torch.int64, device=DEVICE)
    return Xt, lens_t


def make_groups_meta(play_df):
    """Return (groups, metas) where groups is list of feature arrays per nfl_id
       and metas contains (game_id, play_id, nfl_id, frame_ids_array)."""
    df = engineer_features(play_df.copy())
    groups, metas = [], []
    for nfl_id, g in df.groupby("nfl_id"):
        g = g.sort_values("frame_id")
        # keep only features needed (ordered)
        feat = g[[c for c in TREE_FEATURE_COLS if c in g.columns]].values.astype(np.float32)
        groups.append(feat)
        metas.append((
            int(g['game_id'].iloc[0]),
            int(g['play_id'].iloc[0]),
            int(nfl_id),
            g['frame_id'].values.astype(int)
        ))
    return groups, metas


# -------------------------
# Model loading functions
# -------------------------
def load_lstm_model(path=LSTM_MODEL_PATH):
    model = EncoderDecoderLSTM(input_dim=len(TREE_FEATURE_COLS))
    if os.path.exists(path):
        try:
            state = torch.load(path, map_location=DEVICE)
            if isinstance(state, dict) and 'state_dict' in state:
                state = state['state_dict']
            model.load_state_dict(state)
        except Exception as e:
            print("⚠ Failed to load LSTM state_dict:", e)
    else:
        print("⚠ LSTM model missing at:", path)
    model.to(DEVICE).eval()
    return model


def load_xgb_model(path=XGB_MODEL_PATH):
    if xgb is None or (not os.path.exists(path)):
        return None
    try:
        booster = xgb.Booster()
        booster.load_model(path)
        return booster
    except Exception:
        return None


def load_lgb_model(path=LGB_MODEL_PATH):
    if lgb is None or (not os.path.exists(path)):
        return None
    try:
        return lgb.Booster(model_file=path)
    except Exception:
        return None


def load_cat_model(path=CAT_MODEL_PATH):
    if cb is None or (not os.path.exists(path)):
        return None
    try:
        m = cb.CatBoostRegressor()
        m.load_model(path)
        return m
    except Exception:
        return None


# 5. predict_play() — Multi-frame inference (LSTM generating absolute preds)

In [7]:
def predict_play(play_input_df, lstm_model, tree_models=None, T_out_default=10):
    """
    Predict positions for one play (all players in play_input_df).
    Returns DataFrame with columns game_id, play_id, nfl_id, frame_id, x, y
    """
    groups, metas = make_groups_meta(play_input_df)
    if len(groups) == 0:
        return pd.DataFrame(columns=["game_id","play_id","nfl_id","frame_id","x","y"])

    Xt, lens = pad_groups(groups)

    # choose T_out (default fallback)
    if 'num_frames_output' in play_input_df.columns:
        try:
            T_out = int(play_input_df['num_frames_output'].dropna().iloc[0])
            if T_out <= 0:
                T_out = T_out_default
        except Exception:
            T_out = T_out_default
    else:
        T_out = T_out_default

    # LSTM predictions
    with torch.no_grad():
        preds = lstm_model(Xt, lens, T_out=T_out)  # (B, T_out, 2)
        preds_np = preds.cpu().numpy()

    # Build LSTM output rows
    rows = []
    for i, meta in enumerate(metas):
        game_id, play_id, nfl_id, _ = meta
        for t in range(preds_np.shape[1]):
            rows.append({
                "game_id": int(game_id),
                "play_id": int(play_id),
                "nfl_id": int(nfl_id),
                "frame_id": int(t+1),
                "x": float(preds_np[i,t,0]),
                "y": float(preds_np[i,t,1])
            })
    lstm_out_df = pd.DataFrame(rows)

    # If no tree residual models -> return LSTM result
    if tree_models is None or all(m is None for m in tree_models.values()):
        return lstm_out_df

    # Build feature matrix (use last observed frame per player)
    df_input = engineer_features(play_input_df.copy())
    last_input_map = {}
    for nfl_id, g in df_input.groupby("nfl_id"):
        last_input_map[int(nfl_id)] = g.sort_values("frame_id").iloc[-1]

    feat_rows = []
    for _, row in lstm_out_df.iterrows():
        base = last_input_map.get(int(row['nfl_id']))
        if base is None:
            feat_rows.append(np.zeros(len(TREE_FEATURE_COLS), dtype=np.float32))
        else:
            feat_rows.append(np.array([base[c] if c in base.index else 0.0 for c in TREE_FEATURE_COLS], dtype=np.float32))
    feat_matrix = np.vstack(feat_rows) if len(feat_rows) > 0 else np.zeros((0, len(TREE_FEATURE_COLS)), dtype=np.float32)

    # Aggregate residuals
    total_residual = np.zeros((len(lstm_out_df), 2), dtype=np.float32)
    for name, m in tree_models.items():
        if m is None: 
            continue
        try:
            if name == 'xgb':
                if isinstance(m, xgb.Booster):
                    dmat = xgb.DMatrix(feat_matrix, feature_names=TREE_FEATURE_COLS)
                    pred_flat = m.predict(dmat)
                else:
                    pred_flat = m.predict(feat_matrix)
            else:
                pred_flat = m.predict(feat_matrix)

            pred_flat = np.asarray(pred_flat)
            if pred_flat.ndim == 1:
                # assume single-output dx -> dy zero
                dx = pred_flat
                dy = np.zeros_like(dx)
                preds_model = np.vstack([dx, dy]).T
            else:
                preds_model = pred_flat.reshape(-1, 2)
            total_residual += ENSEMBLE_WEIGHTS.get(name, 0.0) * preds_model
        except Exception as e:
            print(f"Warning: tree model {name} failed to predict: {e}")

    # Apply residuals
    final_rows = []
    for i, row in lstm_out_df.iterrows():
        x_final = float(np.clip(row['x'] + total_residual[i,0], 0.0, 120.0))
        y_final = float(np.clip(row['y'] + total_residual[i,1], 0.0, 53.3))
        final_rows.append({
            "game_id": int(row['game_id']),
            "play_id": int(row['play_id']),
            "nfl_id": int(row['nfl_id']),
            "frame_id": int(row['frame_id']),
            "x": x_final,
            "y": y_final
        })
    return pd.DataFrame(final_rows)


def predict_one_play(test_play_df, models):
    """Wrapper expecting a single-play DataFrame (frames of that play)."""
    lstm_model = models.get('lstm')
    if lstm_model is None:
        raise ValueError("LSTM model is required in models dict.")
    tree_models = {k: models.get(k) for k in ['xgb','lgb','cat']}
    preds = predict_play(test_play_df, lstm_model, tree_models)
    return preds[['game_id','play_id','nfl_id','frame_id','x','y']]

# 7. kaggle_main() — evaluation loop

In [8]:
# -------------------------
# Kaggle server-style predict(test, test_input) - required for evaluator
# -------------------------
# We'll implement a `predict` function that works with polars or pandas inputs.
# The evaluator passes:
#   test: DataFrame listing rows to predict (game_id, play_id, nfl_id, frame_id)
#   test_input: DataFrame containing input frames for each play (pre-pass frames)
#
# This function loads models on first call and returns predictions aligned to 'test' rows.
_models_loaded = False
_models_cache = None

def _load_models_once():
    global _models_loaded, _models_cache
    if _models_loaded:
        return
    models = {}
    print("[predict] loading models...")
    models['lstm'] = load_lstm_model(LSTM_MODEL_PATH)
    models['xgb'] = load_xgb_model(XGB_MODEL_PATH)
    models['lgb'] = load_lgb_model(LGB_MODEL_PATH)
    models['cat'] = load_cat_model(CAT_MODEL_PATH)
    _models_cache = models
    _models_loaded = True
    print("[predict] models loaded.")

def predict(test, test_input):
    """
    Kaggle evaluation API signature: predict(test, test_input)
    Accepts polars.DataFrame or pandas.DataFrame. Returns a polars.DataFrame or pandas DataFrame
    with columns ['x','y'] aligned in the SAME order as the `test` argument.
    """
    # lazy model load
    if not _models_loaded:
        _load_models_once()
    models = _models_cache

    # convert polars -> pandas if needed
    try:
        import polars as pl
        is_polars = isinstance(test, pl.DataFrame)
    except Exception:
        is_polars = False

    if is_polars:
        test_pd = test.to_pandas()
        test_input_pd = test_input.to_pandas()
    else:
        test_pd = test.copy()
        test_input_pd = test_input.copy()

    # Build mapping of play inputs (grouped by game_id, play_id)
    input_groups = {k: g.sort_values('frame_id') for k,g in test_input_pd.groupby(['game_id','play_id'])}

    # We'll produce predictions for every play present in test (which lists rows)
    # For efficiency, iterate unique plays referenced in `test_pd`
    preds_rows = []
    # precompute sample_pred rows order so we return predictions aligned to test order
    # test contains multiple rows per play (game_id, play_id, nfl_id, frame_id)
    test_indexed = test_pd.reset_index(drop=True)
    test_indexed['_row_idx'] = np.arange(len(test_indexed))

    # collect predicted outputs per play into a dict for quick lookup
    play_pred_map = {}  # key: (game_id, play_id) -> dataframe of predictions

    for (gid, pid), group in test_pd.groupby(['game_id','play_id']):
        key = (int(gid), int(pid))
        if key not in input_groups:
            # No input frames available for this play (rare). We'll output zeros for these rows later.
            play_pred_map[key] = None
            continue
        play_input_df = input_groups[key]
        # predict for this play (this returns predictions for all nfl_ids in play_input_df)
        try:
            out_df = predict_one_play(play_input_df, models)
            # out_df uses frame_id starting at 1..T_out - ensure dtype alignment
            play_pred_map[key] = out_df
        except Exception as e:
            print(f"[predict] Warning: prediction failed for play {key}: {e}")
            play_pred_map[key] = None

    # Now build a list of predictions aligned to 'test_pd' rows
    pred_x = np.zeros(len(test_pd), dtype=np.float32)
    pred_y = np.zeros(len(test_pd), dtype=np.float32)

    for idx, row in test_pd.reset_index(drop=True).iterrows():
        gid = int(row['game_id'])
        pid = int(row['play_id'])
        nid = int(row['nfl_id'])
        fid = int(row['frame_id'])
        key = (gid, pid)
        out_df = play_pred_map.get(key)
        if out_df is None or len(out_df) == 0:
            # fallback: use last observed x,y for nfl_id in input if available, else 0.0
            try:
                input_play = input_groups[key]
                last = input_play[input_play['nfl_id'] == nid].sort_values('frame_id')
                if len(last) > 0:
                    last_pos = last.iloc[-1]
                    pred_x[idx] = float(last_pos.get('x', 0.0))
                    pred_y[idx] = float(last_pos.get('y', 0.0))
                else:
                    pred_x[idx] = 0.0
                    pred_y[idx] = 0.0
            except Exception:
                pred_x[idx] = 0.0
                pred_y[idx] = 0.0
        else:
            # find row in out_df for same nfl_id and frame_id
            sel = out_df[(out_df['nfl_id'] == nid) & (out_df['frame_id'] == fid)]
            if len(sel) == 0:
                # maybe prediction uses different frame indexing; try closest frame
                sel_close = out_df[(out_df['nfl_id'] == nid)]
                if len(sel_close) > 0:
                    # pick row with same index order: try t-th predicted frame where t = fid-1 if available else last
                    # safe approach: use last predicted frame for this nfl_id
                    srow = sel_close.sort_values('frame_id').iloc[-1]
                    pred_x[idx] = float(srow['x'])
                    pred_y[idx] = float(srow['y'])
                else:
                    pred_x[idx] = 0.0
                    pred_y[idx] = 0.0
            else:
                pred_x[idx] = float(sel.iloc[0]['x'])
                pred_y[idx] = float(sel.iloc[0]['y'])

    # Build DataFrame to return in same order as input 'test'
    out_df = pd.DataFrame({"x": pred_x, "y": pred_y})

    # Convert back to Polars if original was polars
    try:
        import polars as pl
        if is_polars:
            return pl.from_pandas(out_df)
    except Exception:
        pass

    return out_df


# -------------------------
# Offline driver to create CSV submission locally
# -------------------------
def kaggle_main_offline(test_input_path, test_csv_path, submission_out_path):
    """
    Offline inference using local CSVs:
      - test_input.csv : frames for inference (play inputs)
      - test.csv : sample predictions (rows we must predict)
    Creates submission_out_path CSV with columns matching test.csv plus x,y.
    """
    print("[offline] loading models...")
    models = {}
    models['lstm'] = load_lstm_model(LSTM_MODEL_PATH)
    models['xgb'] = load_xgb_model(XGB_MODEL_PATH)
    models['lgb'] = load_lgb_model(LGB_MODEL_PATH)
    models['cat'] = load_cat_model(CAT_MODEL_PATH)
    print("[offline] models ready.")

    print("[offline] reading test_input.csv ...")
    df_input = pd.read_csv(test_input_path)
    print(f"[offline] test_input rows: {len(df_input)}")

    print("[offline] reading test.csv ...")
    sample_spec = pd.read_csv(test_csv_path)
    print(f"[offline] test spec rows: {len(sample_spec)}")

    # Build mapping of play -> predictions (cache)
    input_groups = {k: g.sort_values('frame_id') for k,g in df_input.groupby(['game_id','play_id'])}

    pred_rows = []
    for (gid, pid), grp in sample_spec.groupby(['game_id','play_id']):
        key = (int(gid), int(pid))
        if key not in input_groups:
            # no input frames for this play -> fill zeros for requested rows
            for _, r in grp.iterrows():
                pred_rows.append({
                    "game_id": r['game_id'],
                    "play_id": r['play_id'],
                    "nfl_id": r['nfl_id'],
                    "frame_id": r['frame_id'],
                    "x": 0.0,
                    "y": 0.0
                })
            continue

        play_input_df = input_groups[key]
        try:
            out_df = predict_one_play(play_input_df, models)
        except Exception as e:
            print(f"[offline] warning: predict failed for play {key}: {e}")
            out_df = pd.DataFrame(columns=["game_id","play_id","nfl_id","frame_id","x","y"])

        # Merge requested rows for this play with predicted rows
        merged = grp.merge(out_df, on=['game_id','play_id','nfl_id','frame_id'], how='left')
        # fill missing with last observed or zero
        for idx, row in merged.iterrows():
            x = row.get('x', np.nan)
            y = row.get('y', np.nan)
            if pd.isna(x) or pd.isna(y):
                # fallback: last observed in play_input_df for this nfl_id
                nid = int(row['nfl_id'])
                last = play_input_df[play_input_df['nfl_id'] == nid].sort_values('frame_id')
                if len(last) > 0:
                    lastpos = last.iloc[-1]
                    x = float(lastpos.get('x', 0.0))
                    y = float(lastpos.get('y', 0.0))
                else:
                    x = 0.0
                    y = 0.0
            pred_rows.append({
                "game_id": int(row['game_id']),
                "play_id": int(row['play_id']),
                "nfl_id": int(row['nfl_id']),
                "frame_id": int(row['frame_id']),
                "x": float(x),
                "y": float(y)
            })

    submission = pd.DataFrame(pred_rows)
    # ensure same ordering/columns as sample_spec
    submission = submission.merge(sample_spec[['game_id','play_id','nfl_id','frame_id']], on=['game_id','play_id','nfl_id','frame_id'], how='right')
    # final fill any remaining NaNs (defensive)
    submission['x'] = submission['x'].fillna(0.0)
    submission['y'] = submission['y'].fillna(0.0)

    submission.to_csv(submission_out_path, index=False)
    print("[offline] submission saved to:", submission_out_path)
    return submission


# -------------------------
# Entry point for local runs
# -------------------------
if __name__ == "__main__":
    # local/offline usage
    TEST_INPUT = "/kaggle/input/nfl-big-data-bowl-2026-prediction/test_input.csv"
    TEST_CSV = "/kaggle/input/nfl-big-data-bowl-2026-prediction/test.csv"
    OUT = "/kaggle/working/submission.csv"

    if (os.path.exists(TEST_INPUT) and os.path.exists(TEST_CSV)):
        kaggle_main_offline(TEST_INPUT, TEST_CSV, OUT)
    else:
        print("Offline test files not found; if running inside Kaggle evaluator, implement `predict(test, test_input)` entrypoint.")

[offline] loading models...
⚠ LSTM model missing at: /kaggle/input/nfl-big-data-bowl-2026-prediction/best_lstm.pth
[offline] models ready.
[offline] reading test_input.csv ...
[offline] test_input rows: 49753
[offline] reading test.csv ...
[offline] test spec rows: 5837
[offline] submission saved to: /kaggle/working/submission.csv
